# Superoperator tools

In this notebook we explore the submodules of `operator_tools` that enable easy manipulation of the various quantum channel representations.

To summarize the functionality:
- vectorization and conversions between different repesentations of quantum channels
- apply quantum operations
- compose quantum operations
- validate that quantum channels are physical
- project unphysical channels to physical channels

## Breif motivation and introduction

Perfect gates in **reversible classical computation** are described by permutation matricies, e.g. the [Toffoli gate](https://en.wikipedia.org/wiki/Toffoli_gate), while the input states are vectors. A noisy classical gate could be modeled as a perfect gate followed by a noise channel, e.g. [binary symmetric channel](https://en.wikipedia.org/wiki/Binary_symmetric_channel), on all the bits in the state vector.

Perfect gates in **quantum computation** are described by unitary matricies and states are described by complex vectors, e.g.
$$
|\psi\rangle = U |\psi_0\rangle.
$$

Modeling **noisy quantum computation** often makes use of [mixed states](https://en.wikipedia.org/wiki/Density_matrix) and quantum operations or quantum noise channels.

Interestingly there are a number of ways to represent quantum noise channels, and depending on your task some can be more convenient than others. The simplest case to illustrate this point is to consider a mixed initial state $\rho$ undergoing unitary evolution

$$ \rho' = U \rho U^\dagger.$$

The fact that the unitary has to act on both sides of the inital state means it is a [*superoperator*](https://en.wikipedia.org/wiki/Superoperator), that is an object that can act on operators like the state matrix. 



It turns out using a special matrix multiiplication identity we can write this as
$$ |\rho'\rangle \rangle = \mathcal U |\rho\rangle\rangle,
$$
where $\mathcal U = U^*\otimes U$ and $|\rho\rangle\rangle =  {\rm vec}(\rho)$. The nice thing about this is it looks like the pure state case. This is because the operator (the state) has become a vector and the superoperator (the left right action of $U$) has become an operator. 



**More information**  
Below we will assume that you are already an expert in these topics. If you are unfamilar with these topics we recomend the following references
- chapter 8 of [Mike_N_Ike] which is on *Quantum noise and quantum operations*. 
- chapter 3 of John Preskill's lecture notes [Physics 219/Computer Science 219](http://www.theory.caltech.edu/people/preskill/ph219/chap3_15.pdf)
- the file in `/docs/Superoperator representations.md` 
- for an intuitive but advanced treatment see [GRAPTN]



[Mike_N_Ike] Quantum Computation and Quantum Information  
             Michael A. Nielsen & Isaac L. Chuang  
             Cambridge: Cambridge University Press (2000)   


[GRAPTN] Tensor networks and graphical calculus for open quantum systems  
         Christopher Wood et al.  
         Quant. Inf. Comp. 15, 0579-0811 (2015)  
         (no DOI)  
         https://arxiv.org/abs/1111.6950  

## Conversion between different descriptions of quantum channels

We intentionally chose not to make quantum channels python objects with methods that would automatically transform between represtations. 

The functions to convert between different representations are called things like `kraus2chi`, `kraus2choi`, `pauli_liouville2choi` etc.

This assumes the user does not do silly things like input a choi matrix to a function `chi2choi`.

In [ ]:
import numpy as np
from pyquil.gate_matrices import I, X, Y, Z, H, CNOT

Define some channels

In [ ]:
def amplitude_damping_kraus(p):
    Ad0 = np.asarray([[1, 0], [0, np.sqrt(1 - p)]])
    Ad1 = np.asarray([[0, np.sqrt(p)], [0, 0]])
    return [Ad0, Ad1]

def bit_flip_kraus(p):
    M0 = np.sqrt(1 - p) * I
    M1 = np.sqrt(p) * X
    return [M0, M1]

Define some states

In [ ]:
one_state = np.asarray([[0,0],[0,1]])
zero_state = np.asarray([[1,0],[0,0]])
rho_mixed = np.asarray([[0.9,0],[0,0.1]])

### vec and unvec 

We can vectorize i.e. `vec` and unvec matricies.

We chose a column stacking convention so that the matrix
$$
A = \begin{pmatrix} 1 & 2\\ 3 & 4\end{pmatrix}
$$
becomes
$$
|A\rangle\rangle = {\rm vec}(A) = \begin{pmatrix} 1\\ 3\\ 2\\ 4\end{pmatrix}.
$$

Let's check that

In [ ]:
from forest.benchmarking.operator_tools import vec, unvec

In [ ]:
A = np.asarray([[1, 2], [3, 4]])

In [ ]:
print(A)
print(" ")
print(vec(A))
print(" ")
print('Does the story check out? ', np.all(unvec(vec(A))==A))

### Kraus to $\chi$ matrix (aka chi or process matrix)

In [ ]:
from forest.benchmarking.operator_tools import kraus2chi

Lets do a unitary gate first, say the Hadamard

In [ ]:
print('The Kraus operator is:\n', np.round(H,3))
print('\n')
print('The Chi matrix is:\n', kraus2chi(H))

Now consider the Amplitude damping channel

In [ ]:
AD_kraus = amplitude_damping_kraus(0.1)

print('The Kraus operators are:\n', np.round(AD_kraus,3))
print('\n')
print('The Chi matrix is:\n', np.round(kraus2chi(AD_kraus),3))

### Kraus to Pauli Liouville aka the "Pauli Transfer Matrix"

In [ ]:
from forest.benchmarking.operator_tools import kraus2pauli_liouville

In [ ]:
Hpaulirep = kraus2pauli_liouville(H)
Hpaulirep

We can visualize this using the tools from the plotting module.

In [ ]:
from forest.benchmarking.plotting.state_process import plot_pauli_transfer_matrix
import matplotlib.pyplot as plt

In [ ]:
f, (ax1) = plt.subplots(1, 1, figsize=(5, 4.2))

plot_pauli_transfer_matrix(Hpaulirep,ax=ax1)

The above figure is a graphical representaiton of:
    
(out operator) = H (in operator) H

Z = H X H  
-Y = H Y H  
X = H Z H  

### Evolving states using quantum channels

In many superoperator representations evolution coresponds to mutiplying the vec'ed state by the superoperator. E.g.

In [ ]:
from forest.benchmarking.superoperator_tools import kraus2superop

zero_state_vec = vec(zero_state)

answer_vec = np.matmul(kraus2superop([H]), zero_state_vec)

print('The vec\'ed answer is', answer_vec)
print('\n')
print('The unvec\'ed answer is\n', np.real(unvec(answer_vec)))
print('\n')
print('Let\'s compare it to the normal calculation\n', H @ zero_state @ H)

For representations with this simple application there are no inbuilt functions in forest benchmarking. 

However applying a channel is more painful in the Choi and Kraus representation.

Consider the amplitude damping channel where we need to perform the following calcualtion to find out put of channel 
$\rho_{out} = A_0 \rho A_0^\dagger + A_1 \rho A_1^\dagger.$
We provide helper functions to do these calculations.

In [ ]:
from forest.benchmarking.operator_tools import apply_kraus_ops_2_state, apply_choi_matrix_2_state, kraus2choi

In [ ]:
apply_kraus_ops_2_state(AD_kraus, one_state)

In the Choi representation we get the same answer:

In [ ]:
AD_choi = kraus2choi(AD_kraus)

apply_choi_matrix_2_state(AD_choi, one_state)

### Compose quantum channels

Composing channels is useful when describing larger circuits. In some representations e.g. in the superoperator  or Liouville representation it is just matrix multiplication e.g.

In [ ]:
from forest.benchmarking.operator_tools import superop2kraus, kraus2superop

In [ ]:
H_super = kraus2superop(H)

H_squared_super = H_super @ H_super

print('Hadamard squared as a superoperator:\n', np.round(H_squared_super,2))

print('\n As a Kraus operator:\n', np.round(superop2kraus(H_squared_super),2))

Composing channels in the Kraus represntaion is more difficult. Consider composing two channels $\mathcal A$ (with Kraus operators $[A_0, A_1]$) and $\mathcal B$ (with Kraus operators $[B_0, B_1]$). The composition is 

$$\begin{align}
\mathcal B(\mathcal A(\rho)) & = \sum_i \sum_j B_j A_i \rho A_i^\dagger B_j^\dagger 
\end{align}
$$

In [ ]:
from forest.benchmarking.operator_tools import compose_channel_kraus, superop2kraus

In [ ]:
BitFlip_kraus = bit_flip_kraus(0.2)

kraus2superop(compose_channel_kraus(AD_kraus, BitFlip_kraus))

This is the same as if we do

In [ ]:
BitFlip_super = kraus2superop(BitFlip_kraus)
AD_super = kraus2superop(AD_kraus)

AD_super @ BitFlip_super

We can also easily compose channels acting on independent spaces.

Consider composing the same two channels as above, $\mathcal A$ and $\mathcal B$. However this time they act on different Hilbert spaces. With respect to the tensor product structure $H_2 \otimes H_1$ the Kraus operators are $[A_0\otimes I, A_1\otimes I]$ and $[I \otimes B_0, I \otimes B_1]$.

In this case the order of the operations commutes 
$$\begin{align}
\mathcal A(\mathcal B(\rho))= \mathcal B(\mathcal A(\rho)) & = \sum_i \sum_j  A_i\otimes B_j \rho A_i^\dagger\otimes B_j^\dagger 
\end{align}
$$

In forest benchmarking you can specify the two channels without the Identity tensored on and it will take care of it for you:

In [ ]:
from forest.benchmarking.operator_tools import tensor_channel_kraus

In [ ]:
np.round(tensor_channel_kraus(AD_kraus,BitFlip_kraus),3)

### Validate quantum channels are physical

When doing process tomography sometimes the estimates returned by various estimation methods can result in unphysical processes.

The functions below can be used to check if the estimates are physical.


As a starting point, we might want to check if a process specified by Kraus operators is valid. 

Unless a process is unitary you need more than one Kraus operator to be a valid quantum operation.

In [ ]:
from forest.benchmarking.operator_tools import kraus_operators_are_valid

kraus_operators_are_valid(AD_kraus[0])

However a full set is valid:

In [ ]:
kraus_operators_are_valid(AD_kraus)

We can also validate other properties of quantum channels such as completely positivity and trace preservation. This is done on the **Choi** representation, so you many need to convert your quantum operation to the Choi representaiton first.


In [ ]:
from forest.benchmarking.operator_tools import (choi_is_unitary, 
                                                choi_is_unital, 
                                                choi_is_trace_preserving, 
                                                choi_is_completely_positive, 
                                                choi_is_cptp)

In [ ]:
# amplitude damping is not unitary
print(choi_is_unitary(AD_choi),'\n')

# amplitude damping is not unital
print(choi_is_unital(AD_choi))

In [ ]:
# amplitude damping is trace preserving (TP)
print(choi_is_trace_preserving(AD_choi),'\n')

# amplitude damping is completely positive (CP)
print(choi_is_completely_positive(AD_choi), '\n')

# amplitude damping is CPTP
print(choi_is_cptp(AD_choi))

### Project unphysical channels to physical channels

When doing process tomography often the estimates returned by maximum likelihood estimation or linear inversion methods can result in unphysical processes.

The functions below can be used to project the unphysical estimates back to physical estimates.

In [ ]:
from forest.benchmarking.operator_tools.project_superoperators import (proj_choi_to_completely_positive,
                                                                       proj_choi_to_trace_non_increasing,
                                                                       proj_choi_to_trace_preserving,
                                                                       proj_choi_to_physical)

In [ ]:
neg_Id_choi = -kraus2choi(I)

In [ ]:
proj_choi_to_completely_positive(neg_Id_choi)

In [ ]:
proj_choi_to_trace_non_increasing(neg_Id_choi)

In [ ]:
proj_choi_to_trace_preserving(neg_Id_choi)

In [ ]:
proj_choi_to_physical(neg_Id_choi)

### Validate operators

A lot of the work in validating the physicality of quantum channels comes down to validating properties of matricies:

In [ ]:
from forest.benchmarking.operator_tools.validate_operator import (is_square_matrix, 
                                                                  is_identity_matrix, 
                                                                  is_idempotent_matrix, 
                                                                  is_unitary_matrix, 
                                                                  is_positive_semidefinite_matrix)

In [ ]:
# a vector is not square
is_square_matrix(np.array([[1], [0]]))

In [ ]:
# NBVAL_RAISES_EXCEPTION
# the line above is for testing purposes, do not remove.

# a tensor is not a matrix

tensor = np.ones(8).reshape(2,2,2)
print(tensor)

is_square_matrix(tensor)

In [ ]:
is_identity_matrix(X)

In [ ]:
projector_zero = np.array([[1, 0], [0, 0]])

is_idempotent_matrix(projector_zero)

In [ ]:
is_unitary_matrix(AD_kraus[0])

In [ ]:
is_positive_semidefinite_matrix(I)